In [1]:
import tensorflow as tf

# Custom Training loops

In addition to using built-in APIs for training & validation, e.g. `model.fit()`, `model.evaluate()`, we can define custom loops from scratch using **eager execution and the `GradientTape` object**.

## 1. Build Model

In [2]:
# reuse the model built with functional API
inputs = tf.keras.Input(shape=(28*28,)) 
x = tf.keras.layers.Dense(256, activation='relu')(inputs)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(10)(x)
outputs = tf.keras.layers.Softmax()(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

## 2. Custom Training Loops

In [19]:
# Prepare the training dataset.
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28*28).astype('float32') / 255
x_test = x_test.reshape(10000, 28*28).astype('float32') / 255

# batch dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(128)

In [20]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=0.5)
# Instantiate a loss function.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [21]:
# Select metrics to measure the loss and the accuracy
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [22]:
# Use tf.GradientTape to train the model
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        # Run the forward pass of the layer.
        # The operations that the layer applies to its inputs are going to be recorded
        # on the GradientTape.
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(inputs, training=True)
        # Compute the loss value for this minibatch.
        loss = loss_fn(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # compute metrics
    train_loss(loss)
    train_accuracy(labels, predictions)

In [23]:
for epoch in range(5):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
   
    # Iterate over the batches of the dataset.
    for x_batch_train, y_batch_train in train_dataset:        
        train_step(x_batch_train, y_batch_train)
        test_step(x_batch_train, y_batch_train)

    
    # display at the end of an epoch
    logs = f'Epoch {epoch+1}, Loss: {train_loss.result()}, Training Accuracy: {train_accuracy.result()*100}'
    print(logs)

Epoch 1, Loss: 1.4689747095108032, Training Accuracy: 99.2699966430664
Epoch 2, Loss: 1.4685983657836914, Training Accuracy: 99.29833221435547
Epoch 3, Loss: 1.4685064554214478, Training Accuracy: 99.3133316040039
Epoch 4, Loss: 1.4682337045669556, Training Accuracy: 99.32833099365234
Epoch 5, Loss: 1.468082308769226, Training Accuracy: 99.3499984741211


## 3. Test Model

In [25]:
# test model
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def test_step(inputs, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(inputs, training=False)
    t_loss = loss_fn(labels, predictions)
    
    # compute loss and accuracy metrics
    test_loss(t_loss)
    test_accuracy(labels, predictions)
    
test_step(x_test, y_test)
print(f'Loss: {test_loss.result()}, Test Accuracy: {test_accuracy.result()*100}')

Loss: 1.4828331470489502, Test Accuracy: 97.88999938964844
